In [7]:
%run ../chck_device.ipynb

Using device: cuda
GeForce GTX 1660 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [21]:
from torch.utils.data import dataloader
import csv
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [9]:
RND_MEAN=0
RND_STD=0.0030
LEARNING_RATE=0.001

In [16]:
def abalone_exec(epoch_count=10, mb_size=10, report = 1):
    load_abalone_dataset()
#     init_model()
    train_and_test(epoch_count, mb_size, report)

In [11]:
def load_abalone_dataset():    
    with open('../../data/abalone/abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows=[]
        for row in csvreader:
            rows.append(row)
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt + output_cnt])
    for n, row in enumerate(rows):
        if row[0] == 'I': data[n, 0] = 1
        if row[0] == 'M': data[n, 1] = 1
        if row[0] == 'F': data[n, 2] = 1
        data[n,3:] = row[1:]

In [18]:
def train_and_test(epoch_count,mb_size,report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    
    for epoch in range(epoch_count):
        losses, accs =[],[]
        
        for n in range(step_count):
            
            train_x,train_y =get_train_data(mb_size,n)
            loss,acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
        if report > 0 and (epoch +1)% report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy = {:5.3f}'.\
                 format(epoch+1,np.mean(losses), np.mean(accs),acc))
            
#     final_acc = run_test(test_x,test_y)
    print('\nFinal Test: final accuracy ={:5.3f}'.format(final_acc))

In [13]:
def arrange_data(mb_size):
    global data, shuffle_map,test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count=int(data.shape[0]*0.8)//mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:,:-output_cnt], test_data[:,-output_cnt:]


def get_train_data(mb_size,nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    
    if nth ==0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
        
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    
    return train_data[:,:-output_cnt], train_data[:,-output_cnt:]

In [19]:
abalone_exec()

NameError: name 'run_train' is not defined